In [9]:
import os, re, math, random
from collections import Counter

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# ----------------------------
# 0) Reproducibility + device
# ----------------------------
SEED = 42
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ----------------------------
# 1) Завантаження датасету
# ----------------------------
CSV_PATH = "news_data.csv"   # файл, який ти завантажила

df = pd.read_csv(CSV_PATH)
print("CSV shape:", df.shape)
print("Columns:", list(df.columns))

# Пошук текстової колонки
CANDIDATE_TEXT_COLS = ["text", "content", "body", "article", "news", "description"]
text_col = None
for c in CANDIDATE_TEXT_COLS:
    if c in df.columns:
        text_col = c
        break

if text_col is None:
    obj_cols = [c for c in df.columns if df[c].dtype == "object"]
    if not obj_cols:
        raise ValueError("No text column found.")
    text_col = obj_cols[0]

print("Using text column:", text_col)

texts_raw = df[text_col].dropna().astype(str).tolist()
print("Raw texts:", len(texts_raw))

# ----------------------------
# 2) Cleaning + chunking
# ----------------------------
UA_LETTERS = "а-яіїєґ"
UA_LETTERS_UP = "А-ЯІЇЄҐ"

def clean_text(t: str) -> str:
    t = t.replace("\u00A0", " ")
    t = re.sub(r"<.*?>", " ", t)
    t = re.sub(r"http\S+|www\.\S+", " ", t)
    t = re.sub(r"@\S+", " ", t)
    t = re.sub(rf"[^ {UA_LETTERS}{UA_LETTERS_UP}’'\-]", " ", t)
    t = t.lower()
    t = re.sub(r"\s+", " ", t).strip()
    return t

cleaned = [clean_text(t) for t in texts_raw]
cleaned = [t for t in cleaned if len(t) >= 40]

def chunk_text(t, chunk_words=80, stride=60):
    words = t.split()
    if len(words) <= chunk_words:
        return [t]
    chunks = []
    i = 0
    while i < len(words):
        part = words[i:i+chunk_words]
        if len(part) >= 30:
            chunks.append(" ".join(part))
        i += stride
    return chunks

texts = []
for t in cleaned:
    texts.extend(chunk_text(t))

# Обмеження для швидшого навчання
texts = texts[:20000]
print("Final training samples:", len(texts))

# ----------------------------
# 3) Vocabulary
# ----------------------------
MAX_VOCAB = 20000
specials = ["[PAD]", "[UNK]", "[BOS]", "[EOS]"]

counter = Counter()
for t in texts:
    counter.update(t.split())

most_common = counter.most_common(MAX_VOCAB - len(specials))
itos_list = specials + [w for w, _ in most_common]
stoi = {w: i for i, w in enumerate(itos_list)}

PAD_IDX = stoi["[PAD]"]
UNK_IDX = stoi["[UNK]"]
BOS_IDX = stoi["[BOS]"]
EOS_IDX = stoi["[EOS]"]

vocab_size = len(stoi)
print("Vocab size:", vocab_size)

# ----------------------------
# 4) Encode + LM pairs
# ----------------------------
MAX_LEN = 96

def encode_with_specials(text):
    ids = [stoi.get(w, UNK_IDX) for w in text.split()]
    ids = ids[:MAX_LEN - 2]
    return [BOS_IDX] + ids + [EOS_IDX]

def make_lm_pair(seq):
    return seq[:-1], seq[1:]

def pad_seq(seq, max_len=MAX_LEN-1):
    seq = seq[:max_len]
    if len(seq) < max_len:
        seq = seq + [PAD_IDX] * (max_len - len(seq))
    return torch.tensor(seq, dtype=torch.long)

encoded = [encode_with_specials(t) for t in texts]
pairs = [make_lm_pair(s) for s in encoded]

X = torch.stack([pad_seq(x) for x, _ in pairs])
Y = torch.stack([pad_seq(y) for _, y in pairs])

class LMDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

dataset = LMDataset(X, Y)

val_size = int(0.1 * len(dataset))
train_size = len(dataset) - val_size
train_ds, val_ds = random_split(dataset, [train_size, val_size],
                                generator=torch.Generator().manual_seed(SEED))

BATCH_SIZE = 64   # ↓ було 128
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE)

# ----------------------------
# 5) Decoder-only Transformer
# ----------------------------
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(0, max_len).unsqueeze(1)
        div = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(pos * div)
        pe[:, 1::2] = torch.cos(pos * div)
        self.register_buffer("pe", pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

class TransformerDecoderLM(nn.Module):
    def __init__(self, vocab_size, emb_dim, pad_idx):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim, padding_idx=pad_idx)
        self.pos = PositionalEncoding(emb_dim)
        layer = nn.TransformerDecoderLayer(
            d_model=emb_dim, nhead=4,
            dim_feedforward=256, batch_first=True
        )
        self.decoder = nn.TransformerDecoder(layer, num_layers=2)
        self.fc = nn.Linear(emb_dim, vocab_size)
        self.pad_idx = pad_idx

    def causal_mask(self, T, device):
        m = torch.triu(torch.ones(T, T, device=device), diagonal=1)
        return m.masked_fill(m == 1, float("-inf"))

    def forward(self, x):
        emb = self.pos(self.emb(x))
        T = x.size(1)
        mask = self.causal_mask(T, x.device)
        out = self.decoder(
            emb, emb,
            tgt_mask=mask,
            tgt_key_padding_mask=(x == self.pad_idx),
            memory_key_padding_mask=(x == self.pad_idx)
        )
        return self.fc(out)

model = TransformerDecoderLM(vocab_size, 128, PAD_IDX).to(device)
print("Trainable params:", sum(p.numel() for p in model.parameters()))

# ----------------------------
# 6) Training
# ----------------------------
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

def run_epoch(model, loader, train=True):
    model.train() if train else model.eval()
    total_loss, total_tokens = 0, 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        logits = model(x)
        B, T, V = logits.shape
        loss = criterion(logits.view(B*T, V), y.view(B*T))
        if train:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        tokens = (y != PAD_IDX).sum().item()
        total_loss += loss.item() * tokens
        total_tokens += tokens
    return total_loss / total_tokens

EPOCHS = 3
for ep in range(1, EPOCHS + 1):
    tr = run_epoch(model, train_loader, True)
    va = run_epoch(model, val_loader, False)
    print(f"Epoch {ep}/{EPOCHS} | train loss/token {tr:.4f} | val {va:.4f}")

# ----------------------------
# 7) Generation
# ----------------------------
itos = {i: w for w, i in stoi.items()}

@torch.no_grad()
def generate(prompt, max_new_tokens=40, temperature=1.1):
    model.eval()
    prompt = clean_text(prompt)
    ids = [BOS_IDX] + [stoi.get(w, UNK_IDX) for w in prompt.split()]
    ids = ids[:MAX_LEN-2]

    for _ in range(max_new_tokens):
        x = torch.tensor([ids], device=device)
        logits = model(x)[0, -1] / temperature
        probs = torch.softmax(logits, dim=-1)
        next_id = torch.multinomial(probs, 1).item()
        ids.append(next_id)
        if next_id == EOS_IDX:
            break

    return " ".join(itos[i] for i in ids if i not in (PAD_IDX, BOS_IDX, EOS_IDX))

print("\n--- GENERATION EXAMPLES ---")
for p in ["сьогодні в україні", "економіка країни", "уряд заявив", "війна триває"]:
    print("\nPROMPT:", p)
    print("GEN   :", generate(p))





Device: cuda
CSV shape: (34130, 5)
Columns: ['дів ""Підлісний""', ' що поряд із ЧАЕС', ' відділяє кілька кілометрів. За словами Геращенка', ' загрози для ЧАЕС і ядерних сховищ немає. Радіаційний фон не змінився."', 'True']
Using text column:  що поряд із ЧАЕС
Raw texts: 34130
Final training samples: 20000
Vocab size: 20000
Trainable params: 5537568


/usr/local/lib/python3.12/dist-packages/torch/nn/functional.py:6044: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Epoch 1/3 | train loss/token 7.1428 | val 6.1574
Epoch 2/3 | train loss/token 5.2500 | val 4.0803
Epoch 3/3 | train loss/token 3.1580 | val 2.2625

--- GENERATION EXAMPLES ---

PROMPT: сьогодні в україні
GEN   : сьогодні в україні в в режимі

PROMPT: економіка країни
GEN   : економіка країни відправив пасажирських зробіть численні тиснути

PROMPT: уряд заявив
GEN   : уряд заявив радник радник оп оп оп оп пише

PROMPT: війна триває
GEN   : війна триває триває триває триває триває триває спочатку триває історію біженцям
